In [22]:
import pandas as pd
import numpy as np
import pywt
from scipy.stats import median_absolute_deviation
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from scipy import stats
from tsfresh import extract_features as ef
from tsfresh import select_features
from tsfresh.utilities.dataframe_functions import impute
import os
from tsfresh.feature_extraction import EfficientFCParameters, MinimalFCParameters
from tsfresh.feature_selection.relevance import calculate_relevance_table
from scipy.stats import mannwhitneyu
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from tsfresh import extract_features, select_features
from tsfresh.feature_extraction import ComprehensiveFCParameters


In [2]:
label_index_ro = {"ofm_dim": 1,
               "ifm_dim": 2,
               "kernel_dim": 3,
               "stride": 4,
               "pad": 5}

In [3]:
label_index_layer_boundary = {"layer":0,
                              "boundary":1}

In [18]:
csv_file = '/home/srchand/Desktop/research/NER_VTA/NER_VTA/data/ro_uart/rcg/csvs/reduced/network_56.csv'

# Read the CSV file into a Pandas DataFrame
data = pd.read_csv(csv_file)

len(data)

5285

In [20]:
windows = []
labels_dict = {'id':[],'labels':[]}
labels = []
window_size=50
overlap=1
for i in range(0, len(data) - window_size + 1, overlap):
    window = data.iloc[i:i + window_size]
    if 'boundary' in window['LAYER_OR_BOUNDARY'].values:
        label = label_index_layer_boundary['boundary']
    else:
        label = label_index_layer_boundary['layer']
    
    window_df = pd.DataFrame(window['RO_VALUE'])
    window_df['id'] = i
    window_df['time'] = range(len(window_df))
    window_df['label'] = label
    windows.append(window_df)
    labels.append(label)
    labels_dict['id'].append(i)
    labels_dict['labels'].append(label)

overall_df = pd.concat(windows)

In [ ]:
extraction_settings = ComprehensiveFCParameters()  # You can customize feature extraction settings
extracted_features = extract_features(overall_df, column_id='label', default_fc_parameters=extraction_settings)


Feature Extraction:   0%|                                 | 0/6 [00:00<?, ?it/s]

,RO_VALUE,id_x,time,id_y,labels
0,20020,0,0,0,0
1,20021,0,1,1,0
1,20021,1,0,1,0
2,20011,0,2,2,0
2,20011,1,1,2,0
...,...,...,...,...,...
5235,20006,5231,4,5235,0
5235,20006,5232,3,5235,0
5235,20006,5233,2,5235,0
5235,20006,5234,1,5235,0


In [ ]:
from tsfresh import extract_features, select_features
from tsfresh.feature_extraction import ComprehensiveFCParameters

extraction_settings = ComprehensiveFCParameters()


In [8]:
scaler = StandardScaler()
extracted_features_scaled = scaler.fit_transform(all_features)


In [9]:
n_components = 20  # Adjust the number of components as needed
pca = PCA(n_components=n_components)
features_pca = pca.fit_transform(extracted_features_scaled)


In [10]:
features_pca

array([[16.55760975, -2.41770462, -2.93085013, ...,  0.64364068,
         1.04021706,  0.41558694],
       [16.38049408, -2.50299014, -2.72857358, ...,  1.12229648,
         1.00438041,  0.69257335],
       [16.3064153 , -2.72346021, -3.13561182, ...,  1.01651531,
         1.09512786,  0.81068303],
       ...,
       [16.95169869, -3.78510724, -1.12383974, ...,  1.89999635,
        -0.47587003,  0.15135406],
       [16.67752716, -4.19042549, -0.48444327, ...,  1.67519967,
        -0.23435013,  0.61413795],
       [16.69535169, -4.31240474, -1.22326901, ...,  1.27599094,
        -0.30322258,  0.40179764]])

In [56]:
label_df = pd.DataFrame(labels_dict)
label_df.set_index('id', inplace=True)

y_label_df = label_df[['labels']]
y_label_s = y_label_df['labels'].squeeze()

rt_label = calculate_relevance_table(all_features, y_label_s, ml_task='classification')

In [101]:
stat, p_value = mannwhitneyu(all_features, y_label_s)

In [109]:
significant_features = []
alpha = 0.001
for feature in all_features.columns:
    try:
        stat, p_value = mannwhitneyu(all_features[feature], y_label_s.values.ravel(), alternative='two-sided')
        if p_value < alpha:
            significant_features.append(feature)
    except ValueError:
        # Handle cases where Mann-Whitney U test cannot be performed
        pass

In [98]:
rt_label[rt_label['p_value'] < 0.001]

,feature,type,p_value,relevant
feature,,,,
"RO_VALUE__agg_linear_trend__attr_""slope""__chunk_len_5__f_agg_""min""","RO_VALUE__agg_linear_trend__attr_""slope""__chun...",real,8.642281e-34,True
"RO_VALUE__linear_trend__attr_""slope""","RO_VALUE__linear_trend__attr_""slope""",real,2.598066e-33,True
"RO_VALUE__agg_linear_trend__attr_""slope""__chunk_len_5__f_agg_""mean""","RO_VALUE__agg_linear_trend__attr_""slope""__chun...",real,2.199119e-32,True
"RO_VALUE__linear_trend__attr_""rvalue""","RO_VALUE__linear_trend__attr_""rvalue""",real,5.671161e-32,True
RO_VALUE__time_reversal_asymmetry_statistic__lag_3,RO_VALUE__time_reversal_asymmetry_statistic__l...,real,7.042228e-32,True
...,...,...,...,...
"RO_VALUE__agg_linear_trend__attr_""slope""__chunk_len_5__f_agg_""max""","RO_VALUE__agg_linear_trend__attr_""slope""__chun...",real,6.569053e-04,True
"RO_VALUE__change_quantiles__f_agg_""mean""__isabs_False__qh_0.6__ql_0.0","RO_VALUE__change_quantiles__f_agg_""mean""__isab...",real,6.934971e-04,True
"RO_VALUE__fft_coefficient__attr_""abs""__coeff_8","RO_VALUE__fft_coefficient__attr_""abs""__coeff_8",real,6.952253e-04,True


In [78]:
selected_features = all_features[sf_label['feature']]
combined_df = selected_features.merge(label_df, left_index=True, right_index=True)

In [91]:
"network_410.csv" in fini

True

In [82]:
def map_label_to_tag(label):
    if label == 0:
        return 'layer'
    else:
        return 'boundary'

df['combined_df'] = df['labels'].apply(map_label_to_tag)

KeyError: 'labels'

In [80]:
new_set = set()
new_set.add(1)

In [115]:
fini = ['network_410.csv', 'network_552.csv', 'network_474.csv', 'network_197.csv', 'network_744.csv', 'network_571.csv', 'network_266.csv', 'network_122.csv', 'network_620.csv', 'network_21.csv', 'network_449.csv', 'network_691.csv', 'network_326.csv', 'network_471.csv', 'network_518.csv', 'network_509.csv', 'network_330.csv', 'network_316.csv', 'network_455.csv', 'network_494.csv', 'network_463.csv', 'network_721.csv', 'network_95.csv', 'network_739.csv', 'network_56.csv', 'network_154.csv', 'network_621.csv', 'network_661.csv', 'network_766.csv', 'network_608.csv', 'network_703.csv', 'network_488.csv', 'network_205.csv', 'network_487.csv', 'network_175.csv', 'network_670.csv', 'network_593.csv', 'network_291.csv', 'network_448.csv', 'network_57.csv', 'network_656.csv', 'network_514.csv', 'network_15.csv', 'network_61.csv', 'network_708.csv', 'network_50.csv', 'network_195.csv', 'network_279.csv', 'network_293.csv', 'network_251.csv', 'network_495.csv', 'network_775.csv', 'network_68.csv', 'network_768.csv', 'network_182.csv', 'network_717.csv', 'network_187.csv', 'network_322.csv', 'network_754.csv', 'network_718.csv', 'network_81.csv', 'network_148.csv', 'network_536.csv', 'network_477.csv', 'network_662.csv', 'network_260.csv', 'network_338.csv', 'network_644.csv', 'network_284.csv', 'network_441.csv', 'network_393.csv', 'network_378.csv', 'network_581.csv', 'network_502.csv', 'network_295.csv', 'network_344.csv', 'network_365.csv', 'network_210.csv', 'network_314.csv', 'network_172.csv', 'network_336.csv', 'network_761.csv', 'network_120.csv', 'network_323.csv', 'network_500.csv', 'network_665.csv', 'network_58.csv', 'network_698.csv', 'network_32.csv', 'network_270.csv', 'network_567.csv', 'network_168.csv', 'network_222.csv', 'network_144.csv', 'network_348.csv', 'network_531.csv', 'network_223.csv']
len(fini)


97

In [15]:
all_extracted_features = pd.DataFrame()

In [17]:
for df, label in zip(windows, labels):
    extracted_features = ef(pd.DataFrame(df), label)
    all_extracted_features = pd.concat([all_extracted_features])

ValueError: A value for column_id needs to be supplied

In [13]:
windows

0         20020
1         20021
2         20011
3         20024
4         20020
          ...  
105315    20020
105316    20020
105317    20016
105318    20000
105319    20027
Name: RO_VALUE, Length: 105320, dtype: int64

In [ ]:
all_features = ef(windows, column_id='id', column_sort='time',n_jobs=6, default_fc_parameters=EfficientFCParameters())

In [139]:
data = []
labels_file = '/home/srchand/Desktop/research/TVM_Intel_Fork/tvm/vta/sri_scripts/dataset/ro_uart/conv_hyp/labels/labels.txt'
dataset_path = '/home/srchand/Desktop/research/TVM_Intel_Fork/tvm/vta/sri_scripts/ro_uart_logs/profiling_data/convs/csvs'
with open(labels_file, 'r') as myfile:
    labels_lines = myfile.readlines()[1:]
    
    
label_file_data = {'id': [], 'ifm_h':[], 'ifm_ch':[], 'k':[], 'p':[], 's':[], 'ch':[]}  
    
y_k=[]
y_p=[]
y_s=[]
y_ch = []
for line in labels_lines:
    line = line.rstrip().split('\t')
    for file_name in os.listdir(dataset_path):
        if line[0] in file_name:
            df = pd.read_csv(os.path.join(dataset_path, file_name))
            z_scores = stats.zscore(df['RO_VALUES'])
            threshold = 20
            df = df[(np.abs(z_scores) <= threshold)]
            df.reset_index(drop=True, inplace=True)
            ifm_dim = line[label_index_ro['ifm_dim']].strip('()').split(',')[1:]
            ofm_dim = line[label_index_ro['ofm_dim']].strip('()').split(',')[1:]

            kernel_dim = line[label_index_ro['kernel_dim']].strip('()').split(',')[0]
            stride = line[label_index_ro['stride']].strip('()').split(',')[0]
            pad = line[label_index_ro['pad']].strip('()').split(',')[0]
            df['id'] = line[0]
            df['time'] = range(len(df))
#             label_file_tmp = {}
#             label_file_data['id'] = line[0]
#             label_file_data['ifm_dim'] = [float(dim) for dim in ifm_dim]
#             label_file_data['kernel_dim'] = float(kernel_dim)
#             label_file_data['stride'] = float(stride)
#             label_file_data['pad'] = float(pad)
            wkl_id = line[0]
#             label_file_data[wkl_id] = {}
#             label_file_data[wkl_id]['ifm_dim'] = [float(dim) for dim in ifm_dim]
#             label_file_data[wkl_id]['kernel_dim'] = float(kernel_dim)
#             label_file_data[wkl_id]['stride'] = float(stride)
#             label_file_data[wkl_id]['pad'] = float(pad)
#             label_file_data.append(label_file_tmp)
            label_file_data['id'].append(wkl_id)
            label_file_data['k'].append(float(kernel_dim))
            label_file_data['p'].append(float(pad))
            label_file_data['s'].append(float(stride))
            label_file_data['ch'].append(float(ofm_dim[1]))
            label_file_data['ifm_h'].append(float(ifm_dim[0]))
            label_file_data['ifm_ch'].append(float(ifm_dim[1]))
#             y.append((float(kernel_dim), float(pad), float(stride)))
            y_k.append(float(kernel_dim))
            y_p.append(float(pad))
            y_s.append(float(stride))
            y_ch.append(float(ofm_dim[1]))
            data.append(df)
            

In [71]:
final_df = pd.concat(data, ignore_index=True)

In [73]:
features = ef(final_df, column_id='id', column_sort='time',default_fc_parameters=EfficientFCParameters())


Feature Extraction: 100%|███████████████████████| 10/10 [04:42<00:00, 28.23s/it]


In [74]:
features = impute(features)

/home/srchand/anaconda3/envs/tvm-build-il-2/lib/python3.8/site-packages/tsfresh/utilities/dataframe_functions.py:198: RuntimeWarning: The columns ['RO_VALUES__friedrich_coefficients__coeff_0__m_3__r_30'
 'RO_VALUES__friedrich_coefficients__coeff_1__m_3__r_30'
 'RO_VALUES__friedrich_coefficients__coeff_2__m_3__r_30'
 'RO_VALUES__friedrich_coefficients__coeff_3__m_3__r_30'
 'RO_VALUES__max_langevin_fixed_point__m_3__r_30'
 'RO_VALUES__query_similarity_count__query_None__threshold_0.0'] did not have any finite values. Filling with zeros.
  warnings.warn(


In [143]:
label_df = pd.DataFrame(label_file_data)

In [144]:
label_df.set_index('id',inplace=True)

In [155]:
label_df

,ifm_h,ifm_ch,k,p,s,ch
id,,,,,,
14_14_480_96_1_1_0_0_1_1,14.0,480.0,1.0,0.0,1.0,96.0
56_56_32_32_1_1_0_0_1_1,56.0,32.0,1.0,0.0,1.0,32.0
8_8_1280_384_1_1_0_0_1_1,8.0,1280.0,1.0,0.0,1.0,384.0
104_104_32_64_3_3_1_1_1_1,104.0,32.0,3.0,1.0,1.0,64.0
14_14_1024_256_1_1_0_0_1_1,14.0,1024.0,1.0,0.0,1.0,256.0
...,...,...,...,...,...,...
28_28_512_512_3_3_1_1_2_2,28.0,512.0,3.0,1.0,2.0,512.0
28_28_144_144_3_3_1_1_2_2,28.0,144.0,3.0,1.0,2.0,144.0
13_13_64_256_1_1_0_0_1_1,13.0,64.0,1.0,0.0,1.0,256.0


In [156]:
y_k_df = label_df[['k']]
y_k_s = y_k_df['k'].squeeze()

y_p_df = label_df[['p']]
y_p_s = y_p_df['p'].squeeze()

y_s_df = label_df[['s']]
y_s_s = y_s_df['s'].squeeze()

y_ch_df = label_df[['ch']]
y_ch_s = y_ch_df['ch'].squeeze()

In [185]:
y_kpsch_df = label_df[['k','p','s','ch']]
y_k_p_s_ch = y_kpsch_df[['k','p','s','ch']].squeeze()

In [191]:
y_kpsch_df.unique()

AttributeError: 'DataFrame' object has no attribute 'unique'

In [194]:
rt_all = calculate_relevance_table(features,y_k_s, ml_task='classification')

In [195]:
rt_all

,feature,type,p_value,relevant
feature,,,,
RO_VALUES__fourier_entropy__bins_10,RO_VALUES__fourier_entropy__bins_10,real,0.000019,False
"RO_VALUES__fft_coefficient__attr_""abs""__coeff_11","RO_VALUES__fft_coefficient__attr_""abs""__coeff_11",real,0.000011,False
RO_VALUES__fourier_entropy__bins_5,RO_VALUES__fourier_entropy__bins_5,real,0.000036,False
"RO_VALUES__fft_coefficient__attr_""abs""__coeff_48","RO_VALUES__fft_coefficient__attr_""abs""__coeff_48",real,0.000048,False
RO_VALUES__fourier_entropy__bins_100,RO_VALUES__fourier_entropy__bins_100,real,0.000080,False
...,...,...,...,...
RO_VALUES__number_crossing_m__m_-1,RO_VALUES__number_crossing_m__m_-1,constant,NaN,False
RO_VALUES__number_crossing_m__m_1,RO_VALUES__number_crossing_m__m_1,constant,NaN,False
RO_VALUES__count_above__t_0,RO_VALUES__count_above__t_0,constant,NaN,False


In [157]:
rt_k = calculate_relevance_table(features,y_k_s)
rt_p = calculate_relevance_table(features,y_p_s)
rt_s = calculate_relevance_table(features,y_s_s)
rt_ch = calculate_relevance_table(features,y_ch_s)

In [176]:
sf_k = rt_k[rt_k['p_value'] < 0.005]
sf_p = rt_p[rt_p['p_value'] < 0.005]
sf_s = rt_s[rt_s['p_value'] < 0.005]
sf_ch = rt_ch[rt_ch['p_value'] < 0.005]

In [177]:
sf_final = set(sf_k['feature']).union(set(sf_p['feature']),set(sf_s['feature']),set(sf_ch['feature']))

In [198]:
sf_final

{'RO_VALUES__abs_energy',
 'RO_VALUES__absolute_sum_of_changes',
 'RO_VALUES__agg_linear_trend__attr_"intercept"__chunk_len_10__f_agg_"max"',
 'RO_VALUES__agg_linear_trend__attr_"intercept"__chunk_len_10__f_agg_"min"',
 'RO_VALUES__agg_linear_trend__attr_"intercept"__chunk_len_10__f_agg_"var"',
 'RO_VALUES__agg_linear_trend__attr_"intercept"__chunk_len_50__f_agg_"max"',
 'RO_VALUES__agg_linear_trend__attr_"intercept"__chunk_len_50__f_agg_"min"',
 'RO_VALUES__agg_linear_trend__attr_"intercept"__chunk_len_50__f_agg_"var"',
 'RO_VALUES__agg_linear_trend__attr_"intercept"__chunk_len_5__f_agg_"min"',
 'RO_VALUES__agg_linear_trend__attr_"intercept"__chunk_len_5__f_agg_"var"',
 'RO_VALUES__autocorrelation__lag_9',
 'RO_VALUES__benford_correlation',
 'RO_VALUES__change_quantiles__f_agg_"mean"__isabs_False__qh_0.6__ql_0.0',
 'RO_VALUES__change_quantiles__f_agg_"mean"__isabs_False__qh_0.8__ql_0.0',
 'RO_VALUES__change_quantiles__f_agg_"mean"__isabs_True__qh_0.6__ql_0.0',
 'RO_VALUES__change_quan

In [179]:
selected_features = features[sf_final]

/tmp/ipykernel_68440/1854958445.py:1: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  selected_features = features[sf_final]


In [182]:
input_dim_df = label_df[['ifm_h','ifm_ch']]


In [196]:
combined_df = selected_features.merge(label_df, left_index=True, right_index=True)
combined_df.values.tolist()

[[1349.340805111996,
  394.3409181376429,
  288.0,
  48261697.0,
  1474.1352455688711,
  0.18423867823606324,
  5758.746587095629,
  9.535145022210608,
  3673.0686466819557,
  2704.9250091585945,
  371.26745268258503,
  3020.540381608659,
  8.412638469284994,
  -536.8299672251046,
  0.024485914655889694,
  23.3735824191,
  0.3000118863663378,
  968.0,
  11.342607751471768,
  0.09989813558212808,
  1131.6073193239743,
  726956.2806826584,
  5680.802575704838,
  8.412638469284994,
  48261697.0,
  1.6740888676206467,
  -0.025346224196498564,
  1291.956662371857,
  371.26745268258503,
  322.0,
  2538.3239959600055,
  0.004692181962479532,
  1033.26485881976,
  2820.785372731488,
  2877.229493257189,
  546.3243551024606,
  0.0999086729922847,
  8.412638469284994,
  0.003447046237965054,
  1566.8747207943925,
  5816.438983640794,
  -0.14585372001348648,
  860.5792661639316,
  553.3913484624242,
  1263.2398503395873,
  0.10001538449147225,
  9.535145022210608,
  13.140989063242985,
  1178.951

In [197]:
combined_df[['k','ch','s','p']].values.tolist()

[[3.0, 64.0, 1.0, 1.0],
 [3.0, 128.0, 1.0, 1.0],
 [1.0, 16.0, 1.0, 0.0],
 [1.0, 32.0, 1.0, 0.0],
 [1.0, 32.0, 2.0, 0.0],
 [3.0, 32.0, 1.0, 1.0],
 [3.0, 32.0, 2.0, 1.0],
 [3.0, 128.0, 1.0, 1.0],
 [1.0, 256.0, 1.0, 0.0],
 [3.0, 384.0, 1.0, 1.0],
 [1.0, 128.0, 1.0, 0.0],
 [3.0, 256.0, 1.0, 1.0],
 [3.0, 512.0, 1.0, 1.0],
 [3.0, 256.0, 1.0, 1.0],
 [1.0, 48.0, 1.0, 0.0],
 [1.0, 64.0, 1.0, 0.0],
 [1.0, 192.0, 1.0, 0.0],
 [3.0, 192.0, 1.0, 1.0],
 [3.0, 1024.0, 1.0, 1.0],
 [1.0, 256.0, 1.0, 0.0],
 [1.0, 64.0, 1.0, 0.0],
 [1.0, 256.0, 1.0, 0.0],
 [3.0, 256.0, 1.0, 1.0],
 [3.0, 64.0, 1.0, 1.0],
 [3.0, 32.0, 1.0, 0.0],
 [1.0, 1024.0, 1.0, 0.0],
 [3.0, 1024.0, 2.0, 1.0],
 [1.0, 2048.0, 2.0, 0.0],
 [1.0, 256.0, 1.0, 0.0],
 [1.0, 512.0, 1.0, 0.0],
 [3.0, 224.0, 1.0, 1.0],
 [3.0, 256.0, 1.0, 1.0],
 [3.0, 288.0, 1.0, 1.0],
 [1.0, 160.0, 1.0, 0.0],
 [3.0, 160.0, 1.0, 1.0],
 [3.0, 320.0, 1.0, 1.0],
 [1.0, 400.0, 1.0, 0.0],
 [1.0, 400.0, 2.0, 0.0],
 [1.0, 48.0, 1.0, 0.0],
 [3.0, 48.0, 1.0, 1.0],
 [3.0, 19